 1. Setup & Installation (if not done)

In [ ]:
!pip install langchain openai pandas chromadb sentence-transformers

 2. Load and Explore Dataset

In [1]:
import pandas as pd

Load dataset

In [2]:
df = pd.read_csv("imdb_movies.csv")

Display basic info

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10178 entries, 0 to 10177
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   names       10178 non-null  object 
 1   date_x      10178 non-null  object 
 2   score       10178 non-null  float64
 3   genre       10093 non-null  object 
 4   overview    10178 non-null  object 
 5   crew        10122 non-null  object 
 6   orig_title  10178 non-null  object 
 7   status      10178 non-null  object 
 8   orig_lang   10178 non-null  object 
 9   budget_x    10178 non-null  float64
 10  revenue     10178 non-null  float64
 11  country     10178 non-null  object 
dtypes: float64(3), object(9)
memory usage: 954.3+ KB


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


 3. Data Preprocessing

In [4]:
# Fill missing values
df = df.fillna("")

Combine relevant columns for vectorization

In [5]:
df["combined_text"] = df.apply(lambda row: f"{row['orig_title']} ({row['date_x']}): {row['genre']}. {row['overview']}. Crew: {row['crew']}. Country: {row['country']}", axis=1)

Create metadata for filtering

In [6]:
from langchain.schema import Document
documents = []
for i, row in df.iterrows():
    metadata = {
        "genre": row["genre"],
        "crew": row["crew"],
        "orig_lang": row["orig_lang"],
        "country": row["country"],
        "date": row["date_x"]
    }
    documents.append(Document(page_content=row["combined_text"], metadata=metadata))

 4. Vectorization

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [8]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/f_/x_935x692sd0rjlzt_54vqrw0000gn/T/ipykernel_31579/2537902316.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/varun/megamind/llm-experiments/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector store setup

In [9]:
vectorstore = Chroma.from_documents(documents, embedding_model, persist_directory="./chroma_store")